#Music generation using Google's Magenta
![Logo](https://raw.githubusercontent.com/tensorflow/magenta/master/magenta-logo-bg.png)

I have worked on this project during my internship at the Insight Centre for Data Analytics of Dublin City University, under the supervision of M. Tomas WARD. 
>
Here are the links to my Linkedin and Github profiles.
>**[Github](https://github.com/AlexandreJup)**

>**[LinkedIn](https://www.linkedin.com/in/alexandre-juppet-11628b153/)**

If you have a question, please feel free to send me a mail to ```alexandre.juppet@mines-ales.org```.


---

>



*Happy composition !*

##Step by step, here is how to use Magenta for music composition

---
I would recommend using a GPU for this project, as it is very needy in computing capacities.
You will need to install the GPU version of tensorflow. Please refer to this [link](https://www.tensorflow.org/install/gpu) and verify that you meet all the requirements. 

You will need to install the GPU version of magenta using a command prompt :

```pip install magenta-gpu```.
>
Also check that  you have the right version of *tensorflow*, you can get the latest version of *tensorflow-gpu* by running

```pip install tensorflow-gpu```.
>
> 
> 
 - First of all, we need to decide what type of music we want to generate.
We will need the **largest database possible**. 
Most models will train on **midifiles**, but please check on the readme file of the model you want to use before hand.


- After creating the database, you will need to split it between training and evaluation, before starting the training. You will have to decide on **hyper parameters *(hparams)***, which define the structure of the neural network you will be using.

You need to decide how many steps you should train your model. Default number is 20 000 steps.






### Notes :


- Most of the model creation is done using bash commands, we will be able to use our model in python script once it's trained.If not specified, the code is to **be used in a prompt**.  
>
- This tutorial is based on the *melody_rnn* model, which is one of the simplest to use. You can only generate monophonic track and only use monophonic tracks for training.
>

- If you wish to use other models, please visit the official github [page](https://github.com/tensorflow/magenta) of this open source project.

After having installed *Magenta*, the first step is to naviguate to your working folder, using ```cd``` command. 
You have to start up a virtual environnment in order to use magenta properly, run  the command  ```source activate magenta```.

## Create a dataset of midifiles


First of all, after getting the midi files, we need to convert it to **NotesSequences** :


In [0]:
INPUT_DIRECTORY=<folder containing MIDI and/or MusicXML files. can have child folders.>

# TFRecord file that will contain NoteSequence protocol buffers.
SEQUENCES_TFRECORD=/tmp/notesequences.tfrecord

convert_dir_to_note_sequences \
  --input_dir=$INPUT_DIRECTORY \
  --output_file=$SEQUENCES_TFRECORD \
  --recursive

Once it's done, let's create the dataset of **NoteSequences** using :

In [0]:
melody_rnn_create_dataset \
--config=<one of 'basic_rnn', 'mono_rnn', 'lookback_rnn', or 'attention_rnn'> \ #You need to choose what type of model config you want
--input=/tmp/notesequences.tfrecord \
--output_dir=/tmp/melody_rnn/sequence_examples \
--eval_ratio=0.20 #usually, we would split with 80% of the dataset to be used for training and the remaining 20% for evaluation and validation

## Training the model

We now can train the model ! Here is the basic command to use, we could add some *-hparams* to it

In [0]:
melody_rnn_train \
--config=attention_rnn \ #you need to keep the same config as the one you used to create the dataset
--run_dir=/tmp/melody_rnn/logdir/run1 \
--sequence_example_file=/tmp/melody_rnn/sequence_examples/training_melodies.tfrecord \
--hparams="batch_size=64,rnn_layer_sizes=[64,64],attn_length=40" \ #default size is 128 batch size and 2 layers of 128 nodes
--num_training_steps=20000

### Important note

If using **attention_rnn**, you should probably change how many previous steps the attention mechanism looks at by changing the `attn_length` hyperparameter.

## Evaluating the model

Our model is now trained !
We can run evaluation jobs to check if our model is efficient or not !

**Don't forget you need to keep the same hyper parameters as your training run**

In [0]:
melody_rnn_train \
--config=attention_rnn \
--run_dir=/tmp/melody_rnn/logdir/run1 \
--sequence_example_file=/tmp/melody_rnn/sequence_examples/eval_melodies.tfrecord \
--hparams="batch_size=64,rnn_layer_sizes=[64,64],attn_length=40" \
--num_training_steps=20000 \
--eval


## Creating a bundle file

Now that the model is trained, we can create a bundle file which will simplify the use of a model

In [0]:
melody_rnn_generate \
--config=attention_rnn \
--run_dir=/tmp/melody_rnn/logdir/run1 \
--hparams="batch_size=64,rnn_layer_sizes=[64,64],attn_length=40" \
--bundle_file=/tmp/attention_rnn.mag \ #define the name of your bundle here, don't forget to use a specific name if you change the hparams
--save_generator_bundle

We can now start creating melodies using the model we just trained.

## Using the model we just trained
Two different ways : bash commands or python script
>
If you decide to run the script in bash, you will have to do so locally using a prompt. 




###Bash script

In [0]:
BUNDLE_PATH=<absolute path of .mag file> #path to the bundle file you created previously
CONFIG=<one of 'basic_rnn', 'lookback_rnn', or 'attention_rnn', matching the bundle>

melody_rnn_generate \
--config=${CONFIG} \
--bundle_file=${BUNDLE_PATH} \
--output_dir=/tmp/melody_rnn/generated \ #name of the directory where the generated melodies will be saved
--num_outputs=10 \ #number of generated melodies
--num_steps=128 \ #duration of the generated melodies
--hparams="batch_size=64,rnn_layer_sizes=[64,64],attn_length=40" \
--primer_midi=<path of .mid file you want to base your creation on>

### Python script
*this allows us to have a visual representation of what we've created using the model*

- If we run the python script in Google Colab, we need to run the next cell. Otherwise, we just need to run the  `import` commands
- If you plan on running the Python script locally,  you might have problems using the ```fluidsynth``` library. The use of this library is only to have



In [2]:
print ('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install magenta==1.1.2
!pip install tensorflow
# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print ('Importing libraries and defining some helper functions...')
from google.colab import files



import magenta.music as mm
import magenta

import tensorflow as tf

print ('🎉 Done!')
print (magenta.__version__ )
print (tf.__version__)

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
     |████████████████████████████████| 1.5MB 2.7MB/s 
     |████████████████████████████████| 26.6MB 1.4MB/s 
     |████████████████████████████████| 2.8MB 43.1MB/s 
     |████████████████████████████████| 942kB 45.1MB/s 
     |████████████████████████████████| 204kB 38.8MB/s 
     |████████████████████████████████| 2.3MB 30.5MB/s 
     |█████████████████████████████

Importing libraries and defining some helper functions...


W0807 10:59:28.695132 140293613983616 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/pipelines/statistics.py:132: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W0807 10:59:30.542354 140293613983616 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/music/note_sequence_io.py:60: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.



🎉 Done!
1.1.2
1.14.0


In [0]:
print( 'Downloading model bundle. This will take less than a minute...')

bundle = mm.sequence_generator_bundle.read_bundle_file('/content/drive/My Drive/EMA/Stage/internship DCU/midifiles/attention_rnn.mag')
#bundle = mm.sequence_generator_bundle.read_bundle_file('/content/melody_rnn.mag')
# Import dependencies.
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2

# Initialize the model.
print ("Initializing Melody RNN...")
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['attention_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

print ('🎉 Done!')

In [0]:
#convert midi files to notesequence
seq=mm.midi_file_to_note_sequence('path/to/midifile')
mm.plot_sequence(seq)
mm.play_sequence(seq,synth=mm.fluidsynth)


Loading BokehJS ...

In [0]:
# Model options. Change these to get different generated sequences! 

input_sequence = seq # change this to teapot if you want
num_steps = 256 # change this for shorter or longer sequences
temperature = 0.7 # the higher the temperature the more random the sequence.

# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

# Ask the model to continue the sequence.
sequence = melody_rnn.generate(input_sequence, generator_options)

  
mm.plot_sequence(sequence, show_figure=True)
mm.play_sequence(sequence,mm.midi_synth.fluidsynth)

Loading BokehJS ...

You can of course save the music that has been generated by using the following script.

In [0]:
#save your creation to the working folder
mm.midi_io.note_sequence_to_midi_file(sequence,'creation.mid')